In [8]:
import numpy as np

class ConvNeuron():
    def __init__(self,kernel_size,padding_size,learning_rate):
        self.kernel_size=kernel_size # kernel_size: n -> means n*n 
        self.padding_size=padding_size # padding_size:n , image width:w , image height:h -> w=w+2n,h=h+2n
        self.padded_input
        self.lr=learning_rate
        self.weight_initialize() # initalize weights and bias
        
    def weight_initialize(self):
        self.Kernel=np.random.random([self.kernel_size,self.kernel_size])# Kernel: [n,n] size np array,randomly initialize
        self.bias=np.random.random(1) # bias : scalar 
        
    def Padding(self,input_data):
        H,W=input_data.shape # H as height of image, W as width of image
        self.input_data=input_data
        x=input_data 
        for i in range(self.padding_size):  #from 0 to self.padding_size
            x=np.insert(x,W+i*2,0,axis=1)  # np.insert(array,index_,valuse,axis_dimension): add 0 to column W+i
            x=np.insert(x,0,0,axis=1)      # as above
            
        
        for j in range(self.padding_size): #from 0 to self.padding_size
            x=np.insert(x,H+j*2,0,axis=0) # add 0 array to row H+j
            x=np.insert(x,0,0,axis=0)     # add 0 array to row 0
            
        self.padded_input=x
        return x
        
        
        
    def Forward(self,input_data): 
        #forward operation of convolutional neuron
        L,W=input_data.shape  # get shap of data: L :length(height), W:width
        k=self.kernel_size    #kernel size
        output=np.zeros([L-k+1,W-k+1])    #declare output size array
        
        #operation of convolution~~~~~~~
        for i in range(L-k+1):            
            for j in range(W-k+1):
                for ii in range(k):
                    for jj in range(k):
                        output[i,j]+=input_data[i+ii,j+jj]*self.Kernel[ii,jj]
        output+=self.bias
        #end of convolution--------
        return output
    
    def Backward(self,errors):
        #errors from downstream layer,add each error,
        E=np.sum(errors,axis=0) #add each error
        #calculate and update weights
        L,W=E.shape
        grid_k=np.zeros((L,W))#restore gradient of kernel errors
        grid_b=E.sum() # bias is the sum of error matrix
        
        #calculate gradient of kernel error
        for i in range(L):
            for j in range(W):
                grid_k[i,j]+=E[i,j]*self.padded_input[i:i+L][j:j+W]
        
        #calculate error back to upstream layer, size is as the input data size 
        # it is more complicated, because not 
        H,N=self.input_data.shape
        grid_E=np.zeros((H,N))
        k=self.kernel_size
        for i in range(H):
            for j in range(N):
                for x in range(k):
                    for y in range(k):
                        if i-x>=0 and j-y>=0 and i-x+(k-1)<=L and j-y+(k-1)<=W:
                            grid_E[i,j]+=E[i-x,j-y]*self.Kernel[x,y]
        
        #update weights
        self.bias-=self.lr*grid_b
        self.Kernel-=self.lr*grid_k
        
        return grid_E
        

    def __call__(self, input_data):
        '''
        it is an useful internal function to make coding simpler.
        e.g.
        no __call__ :
        
        a=ConvNeuron(5,5)
        x=a.operationA(input)
        x=a.operationB(x)
        x=a.operationC(x)
        
        with __call__ :
        first define __call__,
        
        def __call__(self,x):
            x=self.operationA(x)
            x=self.operationB(x)
            x=self.operationC(x)
            return x
        
        a=ConvNeuron(5,5)
        x=a(input)       
        '''
        #return self.Forward(input_data)
        x=self.Padding(input_data)
        x=self.Forward(x)
        return x
    
    
    

In [7]:
class PoolingNeuron():
    def __init__(self,learning_rate):
        self.weight_initialize()
        self.lr=learning_rate
    def weight_initialize(self):
        self.bias=np.random.random(1)# only bias here
        
    def Forward(self,input_data):
        L,W=input_data.shape   #get shape of data L:length, W:width
        a=int(L/2)
        b=int(W/2)
        output=np.zeros([a,b]) # create zero array of size[a,b]
        
        #pooling operation----
        for i in range(a):
            for j in range(b):
                output[i,j]=np.mean([input_data[i*2,j*2:j*2+1],input_data[i*2+1,j*2:j*2+1]])
        output+=self.bias
        #end of pooling operation----
        
        return output
    
    def Backward(self,errors):
        #errors from downstream layer,add each error,
        E=np.sum(errors,axis=0) #add each error
        #calculate and update weights
        L,W=E.shape
        grid_b=E.sum() # bias is the sum of error matrix
        
        H,N=self.input_data.shape
        
        grid_E=np.zeros((H,N))
        for i in range(H):
            for j in range(N):
                grid_E=E[i/2,j/2]/4
        
        self.bias-=self.lr*grid_b
        return grid_E
        
        
        
    
    def __call__(self,input_data):
        return self.Forward(input_data)


b=PoolingNeuron(0.02)
x=np.random.random([6,6])
print(x)
y=b(x)
print(y)

[[5.45213594e-01 8.70932158e-01 4.60712836e-02 6.04758485e-01
  6.26508387e-01 1.35672658e-01]
 [4.98523080e-01 9.08997436e-01 8.79399392e-01 7.35847317e-01
  2.86887415e-01 5.71218045e-02]
 [4.54462515e-01 4.23062327e-04 1.56329115e-02 7.78179040e-01
  2.60473090e-01 1.21192102e-02]
 [5.98186068e-01 3.25081532e-01 9.91946803e-01 2.68192256e-01
  6.00901554e-01 8.53186507e-01]
 [3.44589111e-01 4.60773897e-01 9.64092084e-01 8.00579432e-01
  9.87519083e-01 9.16302623e-01]
 [9.20849398e-01 5.53985826e-01 6.33715851e-02 9.51130779e-01
  2.21202090e-01 4.36522696e-01]]
[[0.55018287 0.49104987 0.48501243]
 [0.55463882 0.53210439 0.45900185]
 [0.66103378 0.54204636 0.63267511]]


In [9]:
class ScalarNeuron():
    def __init__(self,input_size,learning_rate):
        #multiple scalar inputs,or see them as plenty of [1x1] array is also ok.
        self.input_size=input_size #define input size
        self.weight_initialize()
        self.lr=learning_rate
        
    def weight_initialize(self):
        self.weights=np.random.random([self.input_size]) # same as input size, or how many scalar inputs
        self.bias=np.random.random(1) # only 1 bias
    
    def Forward(self,input_data):        
        y=np.multiply(input_data,self.weights) #elementwise multiply output size is [self.input_size]
        y=np.sum(y)+self.bias # output is scalar.
        return y
    def Backward(self,errors):
        E=np.sum(errors,axis=0) #add each error
        grid_w=np.ones(self.weights.shape)
        grid_b=1.0
        grid_E=np.zeros(self.input_size)
        
        for i in range(self.input_size):
            grid_E[i]=self.weights[i]
        
        #update weights
        self.bias-=self.lr*grid_b
        self.weights-=self.lr*grid_w
        
        return grid_E
            
        
        
        
        
    
    def __call__(self,input_data):
        return self.Forward(input_data)

Size=10    
L=ScalarNeuron(Size,0.02)
x=np.random.random([Size])
print(x)
y=L(x)
print(y)

[0.55734612 0.14214896 0.9368311  0.82243537 0.20284664 0.59177475
 0.89607881 0.20989387 0.19735711 0.12439078]
[2.43029819]
